In [1]:
from nifti_utils.NiftiHandlerCustom import NiftiHandlerDjangoS3 as nh
# from nifti_utils.AtlasUtils import CustomAtlasLabeler
from nifti_utils.AnalysisUtils import ImageComparison, GroupAnalysis

# s3_path1 = "uploads/lesion_files/AliceWonderlandLesionNetwork_Hong_2010_Case01.nii.gz"
# s3_path2 = "uploads/lesion_files/AliceWonderlandLesionNetwork_Kim_2006_Case01.nii.gz"
# local_path = r"C:\Users\josep\a_repos\Takotsubo\PALM_data\all_vs_symptoms_vox_tstat.nii"
path = r"C:\Users\josep\a_repos\takotsubo\ts_top_lesions\sub-66karnatovskaia2012_space-2mm_lesionMask.nii.gz"

img = nh.load(path)
img.center_of_mass

# img.anatomical_name_at_center_of_mass

(25.22434367541766, 27.517899761336515, 43.95942720763723)

In [10]:
emotion_regulation1 = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\Figure_3C.nii"
emotion_regulation2 = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\Figure_4A.nii"
medulla_func_t = r"C:\Users\josep\a_repos\Takotsubo\emotion_regulation_data\precom_t_medulla_seed.nii.gz"

data = [
    {'path':emotion_regulation1, 'symptom':'emotion_regulation1'},
    {'path':emotion_regulation2, 'symptom':'emotion_regulation2'},
    {'path':medulla_func_t, 'symptom':'medulla_func_t'}
]

df1, df2 = GroupAnalysis(data).cross_correlation_analysis()

display(df1)


c:\Users\josep\miniconda3\envs\ni_env\Lib\site-packages\numpy\core\_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\josep\miniconda3\envs\ni_env\Lib\site-packages\numpy\core\_methods.py:198: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,correlation,path1,path2,symptom1,symptom2
0,-0.212197,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation1,emotion_regulation2
1,-0.134790,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation1,medulla_func_t
2,0.033760,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,C:\Users\josep\a_repos\Takotsubo\emotion_regul...,emotion_regulation2,medulla_func_t


### Topological Analysis: Find Local Extrema

In [6]:
from nifti_utils.NiftiHandlerCustom import NiftiHandler as nh
import pandas as pd

csv_path = r"/PHShome/jt041/projects/takotsubo/csv_data/palm_results_summary.csv"
df_palm_results = pd.read_csv(csv_path)
path = df_palm_results.loc[2,'_vox_tstat.nii']
path = '/PHShome/jt041/projects/takotsubo/' + path
img = nh.load(path)
df = img.get_local_extrema()
df

,x,y,z,value
0,45,45,9,14.203006
1,45,24,9,12.910142
2,57,49,47,9.234034
3,35,51,47,9.192351
4,61,64,45,8.944357
5,32,58,32,7.784228
6,70,19,50,7.380409
7,38,64,66,6.186340
8,18,22,50,4.530478
9,43,11,47,-3.151793


In [7]:
import pandas as pd
df['coord_id'] = df['x'].astype(str) + '_' + df['y'].astype(str) + '_' + df['z'].astype(str)
df['path'] = '/data/nimlab/precomputed_connectomes/GSP1000_MF/' + df['coord_id'] + '_T.nii.gz'
df['image'] = df['path'].apply(nh.load)

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Extract data arrays
X = np.vstack(df['image'].apply(lambda x: x.data.flatten())).T  # Features from df_analysis
y = img.data.flatten()

# Replace NaNs with 0
X = np.nan_to_num(X)
y = np.nan_to_num(y)

X.shape # Returns (902629,31)
X = np.concatenate((X[:, :10], X[:, -5:]), axis=1)
y.shape # Returns (902629,)

# Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

# Initialize and train the linear regression model
model = LinearRegression()
model.fit(X, y)

# # Predict the target for the testing set
# y_pred = model.predict(X_test)

# # Calculate and print the mean squared error (MSE) on the testing set
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error: {mse}')

# You might also want to check the coefficient of determination (R^2) for the model
r_squared = model.score(X, y)
print(f'R^2: {r_squared}')


R^2: 0.8118000242201285


In [28]:
model

LinearRegression()